In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

# Define the folder path
folder_path = '/content/drive/MyDrive/Colab Notebooks/5002/data/Data_Q1/train'

# Get the paths of all files in the folder
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]

# Create an empty DataFrame to store all the data
all_data = pd.DataFrame()

# Read and concatenate files one by one into all_data
for file_path in file_paths:
    # Read the file data
    data = pd.read_csv(file_path)

    # Replace "train" with "data" in the file name as the new column name
    file_name = os.path.basename(file_path)
    data_column = file_name.replace('train', 'data')

    # Add the new column to the data
    data['file'] = data_column

    # Concatenate to all_data
    all_data = pd.concat([all_data, data], ignore_index=True)

In [ ]:
all_data

,x,y,z,a,b,c,d,Is_Falling,file
0,18.495860,13.766527,14.362624,0,0,0,1,0,data_0.csv
1,18.501072,13.827225,14.270268,0,0,1,0,0,data_0.csv
2,18.405950,13.868976,14.094804,1,0,0,0,0,data_0.csv
3,18.444572,13.910701,14.116078,0,1,0,0,0,data_0.csv
4,18.418470,13.933917,14.320566,0,0,0,1,0,data_0.csv
...,...,...,...,...,...,...,...,...,...
134224,23.367303,15.457298,11.470211,0,0,1,0,0,data_19.csv
134225,23.345435,15.475442,11.314662,0,1,0,0,0,data_19.csv
134226,23.323040,15.439008,11.634412,0,0,0,1,0,data_19.csv
134227,23.287241,15.447851,11.501362,0,0,1,0,0,data_19.csv


## smote

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
train_data=all_data
train, valid = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
from collections import Counter
# Use Counter to count the occurrences of each label
label_counts = Counter(train['Is_Falling'])

# Calculate label count
for Label, count in label_counts.items():
    print(f"train Label: {Label}, Count: {count}")

# Calculate label percentages
total_samples = len(train)
label_percentages = {label: count/total_samples for label, count in label_counts.items()}

# Print label percentages
for label, percentage in label_percentages.items():
    print(f"train Label: {label}, Percentage: {round(percentage * 100, 2)}%")

train Label: 0, Count: 102106
train Label: 1, Count: 5277
train Label: 0, Percentage: 95.09%
train Label: 1, Percentage: 4.91%


In [ ]:
from collections import Counter
# Use Counter to count the occurrences of each label
label_counts = Counter(valid['Is_Falling'])

# Calculate label count
for Label, count in label_counts.items():
    print(f"valid Label: {Label}, Count: {count}")

# Calculate label percentages
total_samples = len(valid)
label_percentages = {label: count/total_samples for label, count in label_counts.items()}

# Print label percentages
for label, percentage in label_percentages.items():
    print(f"valid Label: {label}, Percentage: {round(percentage * 100, 2)}%")

valid Label: 0, Count: 25550
valid Label: 1, Count: 1296
valid Label: 0, Percentage: 95.17%
valid Label: 1, Percentage: 4.83%


In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=5002)
X_train = train.drop(['Is_Falling', 'file'], axis=1)
y_train = train['Is_Falling']
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
print("train_resampled Label: 0, Count: %s" % sum(y_train_resampled==0))
print("train_resampled Label: 1, Count: %s" % sum(y_train_resampled==1))

train_resampled Label: 0, Count: 102106
train_resampled Label: 1, Count: 102106


In [ ]:
model = RandomForestClassifier(n_estimators=200,max_depth=5,
                             min_samples_split=5,
                             min_samples_leaf=5,
                             random_state=1)
model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=5,
                       n_estimators=200, random_state=1)

In [ ]:
X_valid = valid.drop(['Is_Falling', 'file'], axis=1)
y_valid = valid['Is_Falling']
y_pred_valid = model.predict(X_valid)
print("Validation Set Evaluation:")
print(classification_report(y_valid, y_pred_valid))

Validation Set Evaluation:
              precision    recall  f1-score   support

           0       0.99      0.60      0.75     25550
           1       0.10      0.92      0.19      1296

    accuracy                           0.62     26846
   macro avg       0.55      0.76      0.47     26846
weighted avg       0.95      0.62      0.72     26846



In [ ]:
test=pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/5002/data/Data_Q1/test/test_set.csv')
X_test = test.drop(['Is_Falling', 'ID'], axis=1)
y_test = test['Is_Falling']

In [ ]:
y_pred_test = model.predict(X_test)
print("test Set Evaluation:")
print(classification_report(y_test, y_pred_test))

test Set Evaluation:
              precision    recall  f1-score   support

           0       0.98      0.74      0.85      6280
           1       0.14      0.74      0.23       343

    accuracy                           0.74      6623
   macro avg       0.56      0.74      0.54      6623
weighted avg       0.94      0.74      0.82      6623



In [ ]:
output = test[['ID']].copy()
output['Is_Falling'] = y_pred_test
#output.to_csv('Q1_output.csv', index=False)

## class weighting

In [ ]:
from collections import Counter
# Use Counter to count the occurrences of each label
label_counts = Counter(train_data['Is_Falling'])

# Calculate label count
for Label, count in label_counts.items():
    print(f"train_data Label: {Label}, Count: {count}")

# Calculate label percentages
total_samples = len(train_data)
label_percentages = {label: count/total_samples for label, count in label_counts.items()}

# Print label percentages
for label, percentage in label_percentages.items():
    print(f"train_data Label: {label}, Percentage: {round(percentage * 100, 2)}%")

train_data Label: 0, Count: 127656
train_data Label: 1, Count: 6573
train_data Label: 0, Percentage: 95.1%
train_data Label: 1, Percentage: 4.9%


In [ ]:
X_train = train_data.drop(['Is_Falling', 'file'], axis=1)
y_train = train_data['Is_Falling']

In [ ]:
clf = RandomForestClassifier(n_estimators=200,class_weight={0: 0.05, 1: 0.95},
                             max_depth=10,
                             min_samples_split=5,
                             min_samples_leaf=5,
                             random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.85      0.91      6280
           1       0.17      0.56      0.27       343

    accuracy                           0.84      6623
   macro avg       0.57      0.71      0.59      6623
weighted avg       0.93      0.84      0.88      6623



In [ ]:
output = test[['ID']].copy()
output['Is_Falling'] = y_pred_test
#output.to_csv('Q1_output.csv', index=False)